In [2]:
import pandas as pd
def calculate_indicator_signals(df: pd.DataFrame, indicator_name, variables, detect_divergence=False):
    signals = {"indicator": indicator_name, "divergence_detected": False, "side": None}

    if indicator_name == "MACD":
        fast_length = variables.get("fast_length", 12)
        slow_length = variables.get("slow_length", 26)
        signal_length = variables.get("signal_length", 9)

        df["EMA_fast"] = df["Close"].ewm(span=fast_length, adjust=False).mean()
        df["EMA_slow"] = df["Close"].ewm(span=slow_length, adjust=False).mean()
        df["MACD_line"] = df["EMA_fast"] - df["EMA_slow"]
        df["MACD_signal"] = df["MACD_line"].ewm(span=signal_length, adjust=False).mean()
        df["MACD_hist"] = df["MACD_line"] - df["MACD_signal"]
        signals["last_value"] = df["MACD_hist"].iloc[-1]

        if detect_divergence:
            if df["MACD_hist"].iloc[-1] > 0 and df["MACD_hist"].iloc[-2] <= 0:
                signals["divergence_detected"] = True
                signals["side"] = "BUY"
            elif df["MACD_hist"].iloc[-1] < 0 and df["MACD_hist"].iloc[-2] >= 0:
                signals["divergence_detected"] = True
                signals["side"] = "SELL"

    elif indicator_name == "RSI":
        length = variables.get("length", 14)
        delta = df["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
        rs = gain / loss
        df["RSI"] = 100 - (100 / (1 + rs))
        signals["last_value"] = df["RSI"].iloc[-1]

        if detect_divergence:
            if df["RSI"].iloc[-1] < 30:
                signals["divergence_detected"] = True
                signals["side"] = "BUY"
            elif df["RSI"].iloc[-1] > 70:
                signals["divergence_detected"] = True
                signals["side"] = "SELL"

    return df, signals

In [6]:

# Assume df is a DataFrame with OHLC data:
# columns: ["Date", "Open", "High", "Low", "Close", "Volume"]
df = pd.read_csv("tests/Historical_data.csv")

# Cleaning and converting the 'Amount' column
for label in ["High", "Low", "Open", "Close"]:
    df[label] = df[label].str.replace('$', '')  # Remove dollar sign
    df[label] = df[label].str.replace(',', '.')  # Replace comma with period
    df[label] = df[label].astype(float)  # Convert to float

# Example: Calculate MACD signals
macd_vars = {
    "fast_length": 12,
    "slow_length": 26,
    "signal_length": 9
}
df, macd_signals = calculate_indicator_signals(df, "MACD", macd_vars, detect_divergence=True)
print("MACD Signals:", macd_signals)



MACD Signals: {'indicator': 'MACD', 'divergence_detected': False, 'side': None, 'last_value': np.float64(-6.722549545264675)}


In [8]:
# Example: Calculate RSI signals without divergence detection
# Assume df is a DataFrame with OHLC data:
# columns: ["Date", "Open", "High", "Low", "Close", "Volume"]
df = pd.read_csv("tests/Historical_data.csv")

# Cleaning and converting the 'Amount' column
for label in ["High", "Low", "Open", "Close"]:
    df[label] = df[label].str.replace('$', '')  # Remove dollar sign
    df[label] = df[label].str.replace(',', '.')  # Replace comma with period
    df[label] = df[label].astype(float)  # Convert to float
rsi_vars = {
    "length": 14
}
df, rsi_signals = calculate_indicator_signals(df, "RSI", rsi_vars, detect_divergence=False)
print("RSI Signals:", rsi_signals)

RSI Signals: {'indicator': 'RSI', 'divergence_detected': False, 'side': None, 'last_value': np.float64(25.879089499877537)}


In [1]:
import pandas as pd

df = pd.read_csv("tests/Historical_data.csv")

# Cleaning and converting the columns into numerical values
for label in ["High", "Low", "Open", "Close"]:
    df[label] = df[label].str.replace('$', '')  # Remove dollar sign
    df[label] = df[label].str.replace(',', '.')  # Replace comma with period
    df[label] = df[label].astype(float)  # Convert to float

df.to_csv("tests/Historical_data_cleaned.csv", index=False)